# Capstone Part One - Dataset Preparation

## Problem Statement
  
The hotel group is facing rising and unpredictable booking cancellations that have led to reduced room utilization and revenue. To address this, management intends to review the current overbooking strategy and has requested analytical support to initiate research into the underlying drivers and patterns of cancellations. This project will leverage data analytics to uncover key factors influencing cancellations and provide insights to guide future pricing and overbooking decisions.

## About the Dataset
- Extraction from hotels’ Property Management System (PMS) SQL databases
- Data collected from two hotels in Portugal  
      
    Hotel 1: Resort region in Algarve   
    Hotel 2: City of Lisbon

- Data is collected dated between 2015 to 2017

## Objective
- To draw insights from available data to show if room pricing is the only factor influencing hotel bookings
- To produce a predictive model to accurately do price benchmarking for the marketing team
- To produce a predictive model to identify potential booking cancellations before working on an overbooking strategy

In [150]:
import pandas as pd
import numpy as np

## Import H1 and H2 Dataset , Deal with Missing Values and Concatenate two datasets

Import and inspect H1.csv belonging to resort hotel

In [151]:
h1_df = pd.read_csv('H1.csv')

In [152]:
h1_df.shape

(40060, 31)

In [153]:
h1_df.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
0,0,342,2015,July,27,1,0,0,2,0,...,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,0,737,2015,July,27,1,0,0,2,0,...,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,0,7,2015,July,27,1,0,1,1,0,...,No Deposit,NULL,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,0,13,2015,July,27,1,0,1,1,0,...,No Deposit,304,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,0,14,2015,July,27,1,0,2,2,0,...,No Deposit,240,NULL,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [154]:
# adding new column 'hotel' to identify the bookings belong to resort hotel
h1_df['hotel'] = 'resort hotel'

In [155]:
# rearranging dataframe for column 'hotel to be first column
h1_df = h1_df[['hotel'] + [col for col in h1_df.columns if col != 'hotel']]

In [156]:
h1_df.head(3)

,hotel,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
0,resort hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,resort hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NULL,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,resort hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NULL,NULL,0,Transient,75.0,0,0,Check-Out,2015-07-02


inpsecting column names for h1_df for analysis

In [157]:
pd.DataFrame(h1_df.columns, columns=['column_names'])

,column_names
0,hotel
1,IsCanceled
2,LeadTime
3,ArrivalDateYear
4,ArrivalDateMonth
5,ArrivalDateWeekNumber
6,ArrivalDateDayOfMonth
7,StaysInWeekendNights
8,StaysInWeekNights
9,Adults


## Missing Values Check

replacing all string datatypes of NULL, Nan and None values to actual nulls to avoid missing detection

In [159]:
h1_df.replace(r'^\s*(NULL|NaN|None|n/a|N/A|nan)?\s*$', np.nan, regex=True, inplace=True)

In [160]:
h1_df_nullcheck = h1_df.isnull().sum()
print("h1_df missing values per column:")
print(h1_df_nullcheck[h1_df_nullcheck>0])

h1_df missing values per column:
Country      464
Agent       8209
Company    36952
dtype: int64



The Country column identifies customer's country of origin.  
464 rows is only less than 1.1 per cent of the entire data set, further check if these unknown country of origin customers contains customers who cancelled their bookings

In [161]:
h1_df[h1_df['Country'].isnull()]['ReservationStatus'].unique()

array(['Check-Out', 'Canceled', 'No-Show'], dtype=object)

checking number of cancelled reservation are there within the country NULL customers

In [162]:
h1_df[(h1_df['Country'].isnull()) & (h1_df['ReservationStatus'] == 'Canceled')].shape[0]

41

checking number of canceled reservations for the entire h1_df

In [163]:
# number of Canceled from the entire dataset
h1_df[(h1_df['ReservationStatus'] == 'Canceled')].shape[0]

10831

checking number of No-Show reservation status within the country NULL customers

In [164]:
# number of No-Show within the null Country group
h1_df[(h1_df['Country'].isnull()) & (h1_df['ReservationStatus'] == 'No-Show')].shape[0]

4

checking number of No-Show reservation status for the entire h1_df 

In [165]:
# number of No-Show from entire dataset
h1_df[(h1_df['ReservationStatus'] == 'No-Show')].shape[0]

291

**Summary of findings:**  
Grouping the data entries by null Country columns  
Total Canceled reservation = 41  
Total No-show reservation = 4  
Total Cancelled and No-Show entries in entire dataset = 11122

**Actions to take:**  
Since the total cancelled and No-show counts within the null country group is less than 1% of the entire dataset, will proceed to drop rows of affected rows as it will not affect the quality of the analysis 

In [168]:
h1_df = h1_df[h1_df['Country'].notnull()]

In [169]:
h1_df_nullcheck = h1_df.isnull().sum()
print("h1_df missing values per column:")
print(h1_df_nullcheck[h1_df_nullcheck>0])

h1_df missing values per column:
Agent       7883
Company    36654
dtype: int64


Agent column represents if the booking was done by a travel agent or not. And having null values is possible if the booking was made directly by the customer.  
Replacing 'Agent' column missing values with placeholder "no agent"

In [173]:
h1_df['Agent'].fillna('No Agent', inplace=True)

The Company column represents the ID of the company/entity that made the booking or responsible for paying the booking. Null values represents bookings not made by the company.  
Replacing 'Company' missing values with placeholder "Not Company"

In [176]:
h1_df['Company'].fillna('Not Company', inplace=True)

C:\Users\yingw\AppData\Local\Temp\ipykernel_39244\2205332203.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  h1_df['Company'].fillna('Not Company', inplace=True)


In [187]:
# final check if any missing values remain
h1_df_nullcheck = h1_df.isnull().sum()
print("h1_df missing values per column:")
print(h1_df_nullcheck[h1_df_nullcheck>0])

h1_df missing values per column:
Series([], dtype: int64)


repeat same steps for H2.csv

Import and inspect H2.csv belonging to city hotel in Lisbon and repeating the steps for resort hotel csv

In [180]:
h2_df = pd.read_csv('H2.csv')

In [203]:
h2_df.shape

(79330, 32)

In [181]:
# adding new column 'hotel' to identify that booking is from city hotel
h2_df['hotel'] = 'city hotel'
h2_df.head()

,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,Children,...,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate,hotel
0,0,6,2015,July,27,1,0,2,1,0.0,...,6,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-03,city hotel
1,1,88,2015,July,27,1,0,4,2,0.0,...,9,NULL,0,Transient,76.5,0,1,Canceled,2015-07-01,city hotel
2,1,65,2015,July,27,1,0,4,1,0.0,...,9,NULL,0,Transient,68.0,0,1,Canceled,2015-04-30,city hotel
3,1,92,2015,July,27,1,2,4,2,0.0,...,9,NULL,0,Transient,76.5,0,2,Canceled,2015-06-23,city hotel
4,1,100,2015,July,27,2,0,2,2,0.0,...,9,NULL,0,Transient,76.5,0,1,Canceled,2015-04-02,city hotel


In [182]:
# rearranging hotel column to be the first column
h2_df = h2_df[['hotel'] + [col for col in h2_df.columns if col != 'hotel']]

In [183]:
h2_df.head()

,hotel,IsCanceled,LeadTime,ArrivalDateYear,ArrivalDateMonth,ArrivalDateWeekNumber,ArrivalDateDayOfMonth,StaysInWeekendNights,StaysInWeekNights,Adults,...,DepositType,Agent,Company,DaysInWaitingList,CustomerType,ADR,RequiredCarParkingSpaces,TotalOfSpecialRequests,ReservationStatus,ReservationStatusDate
0,city hotel,0,6,2015,July,27,1,0,2,1,...,No Deposit,6,NULL,0,Transient,0.0,0,0,Check-Out,2015-07-03
1,city hotel,1,88,2015,July,27,1,0,4,2,...,No Deposit,9,NULL,0,Transient,76.5,0,1,Canceled,2015-07-01
2,city hotel,1,65,2015,July,27,1,0,4,1,...,No Deposit,9,NULL,0,Transient,68.0,0,1,Canceled,2015-04-30
3,city hotel,1,92,2015,July,27,1,2,4,2,...,No Deposit,9,NULL,0,Transient,76.5,0,2,Canceled,2015-06-23
4,city hotel,1,100,2015,July,27,2,0,2,2,...,No Deposit,9,NULL,0,Transient,76.5,0,1,Canceled,2015-04-02


Inspecting column names for h2_df for analysis

In [184]:
pd.DataFrame(h2_df.columns, columns=['column_names'])

,column_names
0,hotel
1,IsCanceled
2,LeadTime
3,ArrivalDateYear
4,ArrivalDateMonth
5,ArrivalDateWeekNumber
6,ArrivalDateDayOfMonth
7,StaysInWeekendNights
8,StaysInWeekNights
9,Adults


checking columns with null values

In [188]:
h2_df.replace(r'^\s*(NULL|NaN|None|n/a|N/A|nan)?\s*$', np.nan, regex=True, inplace=True)

In [190]:
# check for missing values
h2_df_nullcheck = h2_df.isnull().sum()
print("h2_df missing values per column:")
print(h2_df_nullcheck[h2_df_nullcheck>0])

h2_df missing values per column:
Children        4
Country        24
Agent        8131
Company     75641
dtype: int64


Will assume null values in 'Children' Columnn as no children. 

In [195]:
h2_df['Children'].fillna(0, inplace=True)

C:\Users\yingw\AppData\Local\Temp\ipykernel_39244\333593163.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  h2_df['Children'].fillna(0, inplace=True)


check how many canceled in the entire h2_df dataset

In [200]:
h2_df.IsCanceled.value_counts()

IsCanceled
0    46228
1    33102
Name: count, dtype: int64

In [ ]:
h2_df[h2_df['Country'].isnull()]['IsCanceled'].value_counts()

IsCanceled
1    22
0     2
Name: count, dtype: int64

Since the numbered of cancelled bookings belonging to the null values in the 'Country' Columns is only 22.  
Will decide to drop all rows with null values in the Country column as it will not affect the quality of the final analysis.   
  
**summary**  
Total canceled : 33102  
total canceled for null country rows: 22

In [204]:
h2_df = h2_df[h2_df['Country'].notnull()]

In [205]:
# using placeholders to replace null values for Agent and Company column, same justification as H1 dataset cleaning
h2_df['Agent'].fillna('No Agent', inplace=True)
h2_df['Company'].fillna('Not Company', inplace=True)

C:\Users\yingw\AppData\Local\Temp\ipykernel_39244\572891235.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  h2_df['Agent'].fillna('No Agent', inplace=True)
C:\Users\yingw\AppData\Local\Temp\ipykernel_39244\572891235.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exampl

In [208]:
# check for missing values
h2_df_nullcheck = h2_df.isnull().sum()
print("h2_df missing values per column:")
print(h2_df_nullcheck[h2_df_nullcheck>0])

h2_df missing values per column:
Series([], dtype: int64)


## checking datatypes before Concating H1 and H2 datasets

In [209]:
h1_df.dtypes

hotel                           object
IsCanceled                       int64
LeadTime                         int64
ArrivalDateYear                  int64
ArrivalDateMonth                object
ArrivalDateWeekNumber            int64
ArrivalDateDayOfMonth            int64
StaysInWeekendNights             int64
StaysInWeekNights                int64
Adults                           int64
Children                         int64
Babies                           int64
Meal                            object
Country                         object
MarketSegment                   object
DistributionChannel             object
IsRepeatedGuest                  int64
PreviousCancellations            int64
PreviousBookingsNotCanceled      int64
ReservedRoomType                object
AssignedRoomType                object
BookingChanges                   int64
DepositType                     object
Agent                           object
Company                         object
DaysInWaitingList        

In [212]:
h2_df.dtypes

hotel                           object
IsCanceled                       int64
LeadTime                         int64
ArrivalDateYear                  int64
ArrivalDateMonth                object
ArrivalDateWeekNumber            int64
ArrivalDateDayOfMonth            int64
StaysInWeekendNights             int64
StaysInWeekNights                int64
Adults                           int64
Children                       float64
Babies                           int64
Meal                            object
Country                         object
MarketSegment                   object
DistributionChannel             object
IsRepeatedGuest                  int64
PreviousCancellations            int64
PreviousBookingsNotCanceled      int64
ReservedRoomType                object
AssignedRoomType                object
BookingChanges                   int64
DepositType                     object
Agent                           object
Company                         object
DaysInWaitingList        

In [ ]:
h2_df['Children'] = h2_df['Children'].astype(int)

In [215]:
h2_df.Children.dtype

dtype('int64')

 Ready to concate Resort Hotel Data and City Hotel Data into a single dataset

In [216]:
h1_h2_df = pd.concat([h1_df, h2_df], ignore_index=True)

In [225]:
nullcheck = h1_h2_df.isnull().sum()
print(nullcheck[nullcheck>0])

Series([], dtype: int64)


Export combined dataset and work from New Workbook

In [226]:
h1_h2_df.to_csv('combinedhotel.csv', index = False)